# **一个Advantage Actor-Critic算法的简单实现**
## **算法概述**
- 采用批输入数据的方式加速训练;
- 批输入的数据包括TD误差以及log概率;
- 采用TD误差进行批处理训练过程及其不稳定，很难训起来。


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [16]:
import platform
if platform.system == "Darwin":
    PYTORCH_ENABLE_MPS_FALLBACK = 1
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

device = "cuda" if torch.cuda.is_available() else "cpu"

In [17]:
class A3CTrainer(nn.Module):
    def __init__(self, env):
        super(A3CTrainer, self).__init__()
        self.state_dim = env.observation_space.shape[0]
        self.action_dim = env.action_space.n
        
        self.create_training_network()
        self.create_training_method()
        
        self.to(device)

    def create_training_network(self):
        self.fc = nn.Linear(self.state_dim, 20)
        self.value = nn.Sequential(self.fc, nn.ReLU(), nn.Linear(20,1))
        self.actor = nn.Sequential(self.fc, nn.ReLU(), nn.Linear(20, self.action_dim))

    def create_training_method(self):
        self.optim = optim.Adam(self.parameters(), lr=0.001)
    
    def choose_action(self, state):
        with torch.no_grad():
            action_logits = self.actor(state)
            action_probs = F.softmax(action_logits, dim=-1)
            action = torch.multinomial(action_probs,1).item()
            return action, action_probs[action]
    
    def sync_grad(self, policy_agent:nn.Module):
        for behavior_param, target_param in zip(policy_agent.parameters(), self.parameters()):
            if target_param.grad is not None:
                break
            target_param.grad = behavior_param.grad
    
    def update_model(self):
        self.optim.step()
        self.optim.zero_grad()
        

In [18]:
class PolicyAgent(nn.Module):
    GAMMA = 0.9
    
    def __init__(self, env):
        super(PolicyAgent, self).__init__()

        self.state_dim = env.observation_space.shape[0]
        self.action_dim = env.action_space.n
        self.create_policy_network()
        self.create_policy_method()
        
        self.state_batch = []
        self.action_batch = []
        self.reward_batch = []
        
        self.to(device)
        
    def create_policy_network(self):
        self.fc = nn.Linear(self.state_dim, 20)
        self.critic = nn.Sequential(self.fc, nn.ReLU(), nn.Linear(20,1))
        self.actor = nn.Sequential(self.fc, nn.ReLU(), nn.Linear(20, self.action_dim))
        
    def create_policy_method(self):
        self.optim = optim.Adam(self.parameters(), lr = 0.001)
    
    def choose_action(self, state):
        with torch.no_grad():
            state = torch.tensor(state, device=device)
            action_logits = self.actor(state)
            action_probs = F.softmax(action_logits,dim=-1)
            action = torch.multinomial(action_probs, 1)
            return action.item()

    def calculate_batch_td_error(self, state_batch:torch.Tensor, reward_batch:torch.Tensor, done:bool):
        value_batch = self.critic(state_batch)
        values = value_batch[:-1,:].squeeze(-1)
        next_values = value_batch[1:,:].squeeze(-1)
        if done:
            next_values[-1] = 0.
        td_errors = reward_batch + self.GAMMA * next_values - values
        return td_errors
    
    def calculate_policy_loss(self, state_batch, action_batch, td_errors):
        action_logits_batch = self.actor(state_batch)
        log_probs = torch.log(F.softmax(action_logits_batch, dim=-1))
        action_log_probs = torch.gather(log_probs,1,action_batch.unsqueeze(-1)).squeeze(-1)
        policy_loss = action_log_probs * td_errors
        return policy_loss
        
    def perceive(self, state, action, reward, next_state):
        if len(self.state_batch) != 0:
            self.state_batch.pop(-1)
        self.state_batch.append(state)
        self.action_batch.append(action)
        self.reward_batch.append(reward)
        self.state_batch.append(next_state)
    
    def calculate_grad(self, done):
        self.optim.zero_grad()
        state_batch = torch.tensor(self.state_batch, device=device)
        action_batch = torch.tensor(self.action_batch, device=device)
        reward_batch = torch.tensor(self.reward_batch, device=device)
        
        td_errors = self.calculate_batch_td_error(state_batch, reward_batch, done)
        
        value_loss = torch.square(td_errors).mean()
        policy_loss = self.calculate_policy_loss(state_batch, action_batch, td_errors.detach()).mean()
        loss = value_loss - policy_loss
        loss.backward()
        
    
    def sync_model(self,model_dict):
        self.load_state_dict(model_dict, strict=False)
        
    def clear_list(self):
        self.state_batch.clear()
        self.action_batch.clear()
        self.reward_batch.clear()
        

In [19]:
import gym
env_name = "CartPole-v1"
env = gym.make(env_name)

policy_agent = PolicyAgent(env)
target_agent = A3CTrainer(env)

In [20]:
def main():
    for episode in range(3000*2):
        state, _ = env.reset()
        for step in range(300):
            action = policy_agent.choose_action(state)
            next_state, reward, done, _, _ = env.step(action)
            reward = -1 if done else 0.01
            policy_agent.perceive(state, action, reward, next_state)
            state = next_state
            if done:
                break
        policy_agent.calculate_grad(done)
        target_agent.sync_grad(policy_agent)
        target_agent.update_model()
        policy_agent.sync_model(target_agent.state_dict())
        policy_agent.clear_list()
        if episode % 100 == 0 and episode !=0:
            total_reward = 0
            for i in range(10):
                state, _ = env.reset()
                for step in range(300):
                    action = policy_agent.choose_action(state)
                    next_state, reward, done, _, _ = env.step(action)
                    total_reward += reward
                    state = next_state
                    if done:
                        break
            print(f"episode {episode} total reward is {total_reward/10}")

In [21]:
if __name__ == "__main__":
    main()

episode 100 total reward is 21.7
episode 200 total reward is 22.0
episode 300 total reward is 24.8
episode 400 total reward is 23.6
episode 500 total reward is 46.3
episode 600 total reward is 37.7
episode 700 total reward is 37.3
episode 800 total reward is 34.3
episode 900 total reward is 43.4
episode 1000 total reward is 47.0
episode 1100 total reward is 46.3
episode 1200 total reward is 63.0
episode 1300 total reward is 56.8
episode 1400 total reward is 44.5
episode 1500 total reward is 39.6
episode 1600 total reward is 50.4
episode 1700 total reward is 48.8
episode 1800 total reward is 43.6
episode 1900 total reward is 58.5
episode 2000 total reward is 75.1
episode 2100 total reward is 53.1
episode 2200 total reward is 55.6
episode 2300 total reward is 64.3
episode 2400 total reward is 65.7
episode 2500 total reward is 53.6
episode 2600 total reward is 75.2
episode 2700 total reward is 60.0
episode 2800 total reward is 77.2
episode 2900 total reward is 68.9
episode 3000 total rewa

## 实验记录 
2023-3-8 \
1. 实现了部分A3C算法，还有获取行为策略的梯度部分以及梯度同步、模型同步部分。 \
2023-3-9 \
1. 实现了剩余的部分。 \
结果：一个行为策略效果还不错，但是发现m1芯片在小模型训练上表现奇差。在mac上要换成cpu训练。